# _Cone search_ in TAP queries

__DESCRIPTION__

Script to describe different ways to execute by TAP a cone search.

@authors Javier Hernandez & Tamara Civera(CEFCA)

### Official ADQL form
Supported by _almost_ all TAP public services.
```sql
SELECT * FROM jplus.FnuDualObj 
WHERE CONTAINS(POINT('ICRS', alpha_j2000, delta_j2000), CIRCLE('ICRS', 339.26709167, 34.41591944, 300 / 3600.0)) = 1
```
Optimized in the CEFCA catalogues, with the use of __Healpix indexes__, for constant cones with radius __less than half degree__

In [1]:
import time
import pyvo as vo

query = """SELECT alpha_j2000, delta_j2000, a_world, b_world, arc_distance(alpha_j2000, delta_j2000, 339.26709167, 34.41591944) as dist 
 FROM jplus.FnuDualObj 
 WHERE CONTAINS(POINT('ICRS', alpha_j2000, delta_j2000), CIRCLE('ICRS', 339.26709167, 34.41591944, 300 / 3600.0)) = 1 
 ORDER BY dist DESC"""

service = vo.dal.TAPService("http://archive.cefca.es/catalogues/vo/tap/jplus-dr2")
start = time.time()
resultset = service.run_sync(query)
end = time.time()
print("Time", end - start, "s")
print(resultset)

Time 0.1919715404510498 s
<Table length=335>
 alpha_j2000   delta_j2000     a_world        b_world           dist      
                                                                          
   float64       float64       float32        float32         float64     
------------- ------------- -------------- ------------- -----------------
339.352464888  34.460375651  0.00012195394 8.5094594e-05   0.0832705676559
339.197897359 34.4765220239   0.0003353856 0.00030721322   0.0832388077942
339.364515343 34.4370838916   9.280519e-05  8.735175e-05   0.0831004347849
339.222492103 34.3415133241  0.00012020747   9.84132e-05   0.0830131114077
339.342730955 34.3612780855  0.00014093515 0.00011771363   0.0829570393725
339.300121384 34.4941876324  0.00010375319 0.00010183678   0.0828714203147
          ...           ...            ...           ...               ...
339.318576072 34.4161403654  0.00011740173 0.00011321382   0.0424729097043
339.308963467 34.4389330081   0.0030960299  0.002767276

### Simple form with our custom function __arc_distance__
Only available at CEFCA.
```sql
SELECT * FROM FnuDualObj 
WHERE ARC_DISTANCE(alpha_j2000, delta_j2000, 339.26709167, 34.41591944) < 300 / 3600.0
```
_Will be optimized soon_ with the use of Healpix indexes (again for constant cones with radius less than half degree)

In [2]:
query = """SELECT alpha_j2000, delta_j2000, a_world, b_world, arc_distance(alpha_j2000, delta_j2000, 339.26709167, 34.41591944) as dist 
 FROM jplus.FnuDualObj 
 WHERE ARC_DISTANCE(alpha_j2000, delta_j2000, 339.26709167, 34.41591944) < 300 / 3600.0
 ORDER BY dist DESC"""

service = vo.dal.TAPService("http://archive.cefca.es/catalogues/vo/tap/jplus-dr2")
start = time.time()
resultset = service.run_sync(query)
end = time.time()
print("Time", end - start, "s")
print(resultset)


Time 28.53099012374878 s
<Table length=335>
 alpha_j2000   delta_j2000     a_world        b_world           dist      
                                                                          
   float64       float64       float32        float32         float64     
------------- ------------- -------------- ------------- -----------------
339.352464888  34.460375651  0.00012195394 8.5094594e-05   0.0832705676559
339.197897359 34.4765220239   0.0003353856 0.00030721322   0.0832388077942
339.364515343 34.4370838916   9.280519e-05  8.735175e-05   0.0831004347849
339.222492103 34.3415133241  0.00012020747   9.84132e-05   0.0830131114077
339.342730955 34.3612780855  0.00014093515 0.00011771363   0.0829570393725
339.300121384 34.4941876324  0.00010375319 0.00010183678   0.0828714203147
          ...           ...            ...           ...               ...
339.318576072 34.4161403654  0.00011740173 0.00011321382   0.0424729097043
339.308963467 34.4389330081   0.0030960299  0.0027672765

# Spatial search with Healpix

In object tables the column __hpix11__ is the pixel number of the Healpix __ORDER 11 NESTED schema__ where the object position lies. We can obtain the objects in an spatial area testing again the pixels that cover the area. Example:

```sql
SELECT * FROM MagABDualObj WHERE hpix11 IN (1233, 1234, 125553, 125554)
```

The library __Healpy__ contains function to compute pixels for __cones__ and __polygons__.

In [3]:
import healpy
import math

ra = 339.26709167 
dec = 34.41591944
sr = 300 / 3600.0
pixels = healpy.query_disc(2**11, healpy.pixelfunc.ang2vec(math.radians(90.0 - dec), math.radians(ra)), math.radians(sr), inclusive=True, nest=True)
print(pixels)

[13847238 13847239 13847241 13847243 13847244 13847245 13847246 13847247
 13847248 13847249 13847250 13847251 13847254 13847255 13847256 13847257
 13847258 13847259 13847260 13847261 13847262 13847263 13847265 13847267
 13847268 13847269 13847270 13847271 13847273 13847276 13847277 13847279
 13847280 13847281 13847282 13847283 13847284 13847285 13847286 13847288
 13847289 13847292]


In [ ]:
def query_pixels(pixels):
    pixels_str = ','.join([str(x) for x in pixels])
    query = """SELECT alpha_j2000, delta_j2000, a_world, b_world, arc_distance(alpha_j2000, delta_j2000, 339.26709167, 34.41591944) as dist 
         FROM jplus.FnuDualObj WHERE hpix11 IN ({0})
         ORDER BY dist DESC""".format(pixels_str)

    service = vo.dal.TAPService("http://10.200.81.13/catalogues/vo/tap/jplus-dr2")
    start = time.time()
    resultset = service.run_sync(query)
    end = time.time()
    print("Time", end - start, "s")
    print(resultset)
    
query_pixels(pixels)

In [ ]:
# Each vertex (90 - dec, ra)
vertex = [(math.radians(90.0 - 1.0), math.radians(1.0)),
          (math.radians(90.0 - 1.0), math.radians(1.01)),
          (math.radians(90.0 - 1.01), math.radians(1.01)),
          (math.radians(90.0 - 1.02), math.radians(.99)),
          (math.radians(90.0 - 1.01), math.radians(.98))]
nvxs = [healpy.pixelfunc.ang2vec(vx[0], vx[1]) for vx in vertex]

pixels = healpy.query_polygon(2**11, nvxs, inclusive=True, nest=True)
print(pixels)

In [ ]:
query_pixels(pixels)